In [31]:
filename = './datasets/spar/spar_dataset'
seed = 25
# read through the folder that end with csv:
import os
import random
import pandas as pd
from sklearn.preprocessing import LabelEncoder
classes = []
subjs = []
for root, dirs, files in os.walk(filename):
    for file in files:
        if file.endswith(".csv") and 'R' in file:
            label = file.split('_')[1]
            subj = file.split('_')[0]
            if label not in classes:
                classes.append(label)
            if subj not in subjs:
                subjs.append(subj)
            # print(os.path.join(root, file))
            # df = pd.read_csv(os.path.join(root, file))

# generate train combo => ABC, ABD, ... # test combo => BCD (some combo that was not seem in train) based on the classes
train_combo = [('E1', 'E2', 'E3')]
test_combo = [('E3', 'E1', 'E2')]

assert all([each in classes for comb in train_combo for each in comb]), 'train combo not in classes'
assert all([each in classes for comb in test_combo for each in comb]), 'test combo not in classes'
train_classes = list(set([each for comb in train_combo for each in comb]))
le = LabelEncoder()
le.fit(train_classes)
random.seed(seed)
train_samples = []
test_samples = []
for each_subj in subjs:
    collection = {class_name: [] for class_name in classes}
    for root, dirs, files in os.walk(filename):
        for file in files:
            if file.endswith(".csv") and 'R' in file and each_subj == file.split('_')[0]:
                label = file.split('_')[1]
                collection[label].append(file)

    lengths = [len(collection[class_name]) for class_name in classes]
    minimum = min(lengths)
    # 80% for train, 20% for test
    train_length = int(minimum * 0.8)
    test_length = minimum - train_length
    print(f'{each_subj} has {minimum} samples, {train_length} for train, {test_length} for test')
    # generate train and test data
    
    # randomly extract one sample of label from the collection and remove that label from the list in the collection:
    for _ in range(train_length):
        for i in train_combo:
            cur_combo = []
            for j in i:
                selected_sample = random.choice(collection[j])
                collection[j].remove(selected_sample)
                cur_combo.append(selected_sample)
            train_samples.append(cur_combo)
    for _ in range(test_length):
        for i in test_combo:
            cur_combo = []
            for j in i:
                selected_sample = random.choice(collection[j])
                collection[j].remove(selected_sample)
                cur_combo.append(selected_sample)
            test_samples.append(cur_combo)
    

S8 has 20 samples, 16 for train, 4 for test
S17 has 19 samples, 15 for train, 4 for test
S13 has 19 samples, 15 for train, 4 for test
S12 has 20 samples, 16 for train, 4 for test
S11 has 20 samples, 16 for train, 4 for test
S20 has 15 samples, 12 for train, 3 for test
S1 has 20 samples, 16 for train, 4 for test
S15 has 20 samples, 16 for train, 4 for test
S6 has 19 samples, 15 for train, 4 for test
S5 has 20 samples, 16 for train, 4 for test
S10 has 20 samples, 16 for train, 4 for test
S2 has 19 samples, 15 for train, 4 for test
S16 has 20 samples, 16 for train, 4 for test
S18 has 20 samples, 16 for train, 4 for test
S4 has 19 samples, 15 for train, 4 for test
S14 has 20 samples, 16 for train, 4 for test
S19 has 20 samples, 16 for train, 4 for test
S3 has 10 samples, 8 for train, 2 for test
S7 has 20 samples, 16 for train, 4 for test
S9 has 20 samples, 16 for train, 4 for test


In [32]:
len(train_samples), len(test_samples)

(303, 77)

In [37]:

import numpy as np


def combo_to_np(combo, le: LabelEncoder):
    samples = []
    labels = []
    for each_combo in combo:
        for each_file in each_combo:
            df = pd.read_csv(os.path.join(filename, each_file))
            samples.append(df.to_numpy()[:, 1:7])
            labels.append([le.transform([each_file.split('_')[1]])[0]] * df.shape[0])
            # print('labels', labels)
    # convert to np:
    samples = np.concatenate(samples)
    labels = np.concatenate(labels)
    print(samples.shape, labels.shape)
    return samples, labels
combo_to_np(train_samples, le)


(112181, 6) (112181,)


(array([[-1.100601,  0.461792, -0.851196,  0.119459, -0.400505, -0.099266],
        [-1.160965,  0.400863, -0.83046 , -0.17802 ,  0.013153,  0.019373],
        [-1.16362 ,  0.484878, -0.711365, -0.156335,  0.423481,  0.204342],
        ...,
        [ 0.368675,  0.7874  ,  0.718854,  0.976343,  1.023894,  0.792925],
        [ 0.403206,  0.739999,  0.770553,  0.689447,  0.591772,  0.616874],
        [ 0.413861,  0.733648,  0.782116,  0.261578,  0.189895,  0.465816]]),
 array([0, 0, 0, ..., 2, 2, 2]))

In [1]:
from utils_loader import test_idea_dataloader_ABC_to_BCA


test_idea_dataloader_ABC_to_BCA({'seed': 25})

S8 has 20 samples, 16 for train, 4 for test
S17 has 19 samples, 15 for train, 4 for test
S13 has 19 samples, 15 for train, 4 for test
S12 has 20 samples, 16 for train, 4 for test
S11 has 20 samples, 16 for train, 4 for test
S20 has 15 samples, 12 for train, 3 for test
S1 has 20 samples, 16 for train, 4 for test
S15 has 20 samples, 16 for train, 4 for test
S6 has 19 samples, 15 for train, 4 for test
S5 has 20 samples, 16 for train, 4 for test
S10 has 20 samples, 16 for train, 4 for test
S2 has 19 samples, 15 for train, 4 for test
S16 has 20 samples, 16 for train, 4 for test
S18 has 20 samples, 16 for train, 4 for test
S4 has 19 samples, 15 for train, 4 for test
S14 has 20 samples, 16 for train, 4 for test
S19 has 20 samples, 16 for train, 4 for test
S3 has 10 samples, 8 for train, 2 for test
S7 has 20 samples, 16 for train, 4 for test
S9 has 20 samples, 16 for train, 4 for test
(112103, 6) (112103,)
(28519, 6) (28519,)


((array([[-1.089524,  0.295365, -0.578415,  0.184561, -0.76863 , -0.247805],
         [-1.099212,  0.35115 , -0.661316, -0.028319, -0.513699, -0.189888],
         [-1.118485,  0.389222, -0.67424 , -0.188227, -0.220624, -0.148195],
         ...,
         [ 0.394497,  0.599451,  0.842365, -0.074083, -0.505722, -0.075523],
         [ 0.391111,  0.615858,  0.847801, -0.223795, -0.689337, -0.187778],
         [ 0.395309,  0.639557,  0.818864, -0.298615, -0.914531, -0.288899]]),
  array([0, 0, 0, ..., 2, 2, 2])),
 (array([[ 0.065735,  1.028625,  0.254135, -0.238591,  0.203898, -0.042967],
         [ 0.084106,  1.038407,  0.236938, -0.136968, -0.077328, -0.048223],
         [ 0.081329,  0.992615,  0.228012, -0.061495, -0.390415, -0.032601],
         ...,
         [-0.936938,  0.46501 , -0.635976,  0.319458, -0.863067, -0.151261],
         [-0.94113 ,  0.316814, -0.714352, -0.799177, -0.724849, -0.141858],
         [-0.942344,  0.240762, -0.778274, -1.839198, -0.123219, -0.011225]]),
  array([